# Edgar company search

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext rich
from rich import inspect

In [2]:
import sys

if not '..' in sys.path: sys.path.append('..')

## Set your edgar identity

In [3]:
from edgar import set_identity
set_identity("Dwight Gunning dgunning@gmail.com")

[07:38:39] INFO     Identity of the Edgar REST client set to [Dwight Gunning dgunning@gmail.com]         ]8;id=999653;file://C:\Users\dwight\PycharmProjects\edgar\notebooks\..\edgar\core.py\core.py]8;;\:]8;id=182043;file://C:\Users\dwight\PycharmProjects\edgar\notebooks\..\edgar\core.py#46\46]8;;\

In [4]:
import os

os.environ['EDGAR_IDENTITY']

'Dwight Gunning dgunning@gmail.com'

## Find a company

In [5]:
from edgar.company import Company, get_company_tickers

### Find a company using a cik

In [6]:
expe = Company.for_cik(1324424)
expe

Expedia Group, Inc. 
 
 
 
 
 CIK 
 Industry 
 Category 
 
 
 
 
 1324424 
 Transportation Services 
 Large accelerated filer 
 
 
 

 
 
 
 
 Exchange 
 Ticker 
 
 
 
 
 Nasdaq 
 EXPE

### Find a company using a ticker

In [7]:
snow = Company.for_ticker("snow")
snow

Snowflake Inc. 
 
 
 
 
 CIK 
 Industry 
 Category 
 
 
 
 
 1640147 
 Services-Prepackaged Software 
 Large accelerated filer 
 
 
 

 
 
 
 
 Exchange 
 Ticker 
 
 
 
 
 NYSE 
 SNOW

In [8]:
inspect(snow)

╭─────────────────────────────── <class 'edgar.company.Company'> ────────────────────────────────╮
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Company(                                                                                   │ │
│ │ │   cik=1640147,                                                                           │ │
│ │ │   name='Snowflake Inc.',                                                                 │ │
│ │ │   tickers=['SNOW'],                                                                      │ │
│ │ │   exchanges=['NYSE'],                                                                    │ │
│ │ │   sic=7372,                                                                              │ │
│ │ │   sic_description='Services-Prepackaged Software',                                       │ │
│ │ │   category='Large accelerated filer',                                                    │ │
│ │ │   fiscal_year_end='0131',                                                                │ │
│ │ │   entity_type='operating',                                                               │ │
│ │ │   phone='844-766-9355',                                                                  │ │
│ │ │   flags='',                                                                              │ │
│ │ │   business_address=Address(                                                              │ │
│ │ │   │   street1='106 EAST BABCOCK STREET',                                                 │ │
│ │ │   │   street2='SUITE 3A',                                                                │ │
│ │ │   │   city='BOZEMAN',                                                                    │ │
│ │ │   │   state_or_country='MT',                                                             │ │
│ │ │   │   zipcode='59715',                                                                   │ │
│ │ │   │   state_or_country_desc='MT'                                                         │ │
│ │ │   ),                                                                                     │ │
│ │ │   mailing_address=Address(                                                               │ │
│ │ │   │   street1='106 EAST BABCOCK STREET',                                                 │ │
│ │ │   │   street2='SUITE 3A',                                                                │ │
│ │ │   │   city='BOZEMAN',                                                                    │ │
│ │ │   │   state_or_country='MT',                                                             │ │
│ │ │   │   zipcode='59715',                                                                   │ │
│ │ │   │   state_or_country_desc='MT'                                                         │ │
│ │ │   ),                                                                                     │ │
│ │ │   filings=Snowflake Inc. 1640147 Filings - 322 in total from 2017-04-11 to 2022-12-14    │ │
│ │ )                                                                                          │ │
│ ╰────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                │
│ business_address = Address(                                                                    │
│                        street1='106 EAST BABCOCK STREET',                                      │
│                        street2='SUITE 3A',                                                     │
│                        city='BOZEMAN',                                                         │
│                        state_or_country='MT',                                                  │
│                        zipcode='59715',                                                        │
│                        state_or_countr

### Ticker symbol

In [9]:
snow.tickers

['SNOW']

### Industry

In [10]:
snow.industry

'Services-Prepackaged Software'

## Company Filings

In [11]:
filings = snow.get_filings(form='10-Q')
filings

Count,Start,End
7,2020-12-03,2022-12-02


### Get the latest filing

In [14]:
snow.get_filings(form='10-Q').latest()

Filing(form='10-Q', company='Snowflake Inc.', cik=1640147, date='2022-12-02', accession_no='0001640147-22-000100')

In [13]:
filings.to_pandas()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001640147-22-000100,2022-12-02,2022-10-31,2022-12-02T16:14:33.000Z,34,10-Q,001-39504,,10945750,1,1,snow-20221031.htm,10-Q
1,0001640147-22-000084,2022-09-01,2022-07-31,2022-09-01T16:08:55.000Z,34,10-Q,001-39504,,10338946,1,1,snow-20220731.htm,10-Q
2,0001640147-22-000044,2022-06-03,2022-04-30,2022-06-03T16:10:48.000Z,34,10-Q,001-39504,,9437050,1,1,snow-20220430.htm,10-Q
3,0001640147-21-000271,2021-12-03,2021-10-31,2021-12-03T16:51:17.000Z,34,10-Q,001-39504,,9775510,1,1,snow-20211031.htm,10-Q
4,0001640147-21-000212,2021-09-02,2021-07-31,2021-09-02T16:58:03.000Z,34,10-Q,001-39504,,8674777,1,1,snow-20210731.htm,10-Q
5,0001640147-21-000122,2021-06-04,2021-04-30,2021-06-04T16:32:41.000Z,34,10-Q,001-39504,,7805314,1,1,snow-20210430.htm,10-Q
6,0001640147-20-000023,2020-12-03,2020-10-31,2020-12-02T19:45:24.000Z,34,10-Q,001-39504,,9433054,1,1,snow-20201031.htm,10-Q


In [15]:
filings[0].file_number

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\dwight\AppData\Local\Temp\ipykernel_24124\75660520.py:1 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\dwight\\AppData\\Local\\Temp\\ipykernel_24124\\75660520.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Filing' object has no attribute 'file_number'

## Company Facts

In [10]:
facts = snow.get_facts()
facts

Company Facts(SNOWFLAKE INC. [1640147] 3,672 total facts)

### Get facts as a DuckDB

In [11]:
db = snow.get_facts().db()

In [12]:
db.execute(
"select fact, val, filed from facts"
).df()

,fact,val,filed
0,EntityCommonStockSharesOutstanding,2.887000e+08,2021-03-31
1,EntityCommonStockSharesOutstanding,2.961000e+08,2021-06-04
2,EntityCommonStockSharesOutstanding,3.009000e+08,2021-09-02
3,EntityCommonStockSharesOutstanding,3.063000e+08,2021-12-03
4,EntityCommonStockSharesOutstanding,3.146000e+08,2022-03-30
...,...,...,...
3667,StockholdersEquityIncludingPortionAttributable...,4.964965e+09,2022-12-02
3668,StockholdersEquityIncludingPortionAttributable...,5.006682e+09,2022-12-02
3669,StockholdersEquityIncludingPortionAttributable...,5.049045e+09,2022-12-02
3670,StockholdersEquityIncludingPortionAttributable...,5.426323e+09,2022-12-02


### Facts to pandas dataframe

In [13]:
facts.to_pandas()

,namespace,fact,val,accn,end,fy,fp,form,filed,frame,start
0,dei,EntityCommonStockSharesOutstanding,2.887000e+08,0001640147-21-000073,2021-03-01,2020,FY,10-K,2021-03-31,CY2021Q1I,None
1,dei,EntityCommonStockSharesOutstanding,2.961000e+08,0001640147-21-000122,2021-05-14,2022,Q1,10-Q,2021-06-04,None,None
2,dei,EntityCommonStockSharesOutstanding,3.009000e+08,0001640147-21-000212,2021-08-20,2022,Q2,10-Q,2021-09-02,CY2021Q3I,None
3,dei,EntityCommonStockSharesOutstanding,3.063000e+08,0001640147-21-000271,2021-11-19,2022,Q3,10-Q,2021-12-03,CY2021Q4I,None
4,dei,EntityCommonStockSharesOutstanding,3.146000e+08,0001640147-22-000023,2022-03-18,2022,FY,10-K,2022-03-30,CY2022Q1I,None
...,...,...,...,...,...,...,...,...,...,...,...
3667,us-gaap,StockholdersEquityIncludingPortionAttributable...,4.964965e+09,0001640147-22-000100,2021-07-31,2023,Q3,10-Q,2022-12-02,CY2021Q2I,None
3668,us-gaap,StockholdersEquityIncludingPortionAttributable...,5.006682e+09,0001640147-22-000100,2021-10-31,2023,Q3,10-Q,2022-12-02,CY2021Q3I,None
3669,us-gaap,StockholdersEquityIncludingPortionAttributable...,5.049045e+09,0001640147-22-000100,2022-01-31,2023,Q3,10-Q,2022-12-02,CY2021Q4I,None
3670,us-gaap,StockholdersEquityIncludingPortionAttributable...,5.426323e+09,0001640147-22-000100,2022-07-31,2023,Q3,10-Q,2022-12-02,CY2022Q2I,None


In [14]:
from edgar.company import Company, get_company_tickers
tickers = get_company_tickers()

In [15]:
tickers.CIK.value_counts().to_frame()

,CIK
312070,41
1114446,29
1026214,24
927971,17
70858,17
...,...
877463,1
1403475,1
1609809,1
1487952,1


In [16]:
Company.for_cik(1832950)

Kernel Group Holdings, Inc. 
 
 
 
 
 CIK 
 Industry 
 Category 
 
 
 
 
 1832950 
 Blank Checks 
 Non-accelerated filer Smaller reporting company Emerging growth company 
 
 
 

 
 
 
 
 Exchange 
 Ticker 
 
 
 
 
 Nasdaq 
 KRNL 
 
 
 Nasdaq 
 KRNLU 
 
 
 Nasdaq 
 KRNLW

In [17]:
Company.for_ticker('SNOW')

Snowflake Inc. 
 
 
 
 
 CIK 
 Industry 
 Category 
 
 
 
 
 1640147 
 Services-Prepackaged Software 
 Large accelerated filer 
 
 
 

 
 
 
 
 Exchange 
 Ticker 
 
 
 
 
 NYSE 
 SNOW

In [18]:
tickers

,CIK,Ticker,Company
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1067983,BRK-B,BERKSHIRE HATHAWAY INC
3,731766,UNH,UNITEDHEALTH GROUP INC
4,200406,JNJ,JOHNSON & JOHNSON
...,...,...,...
11606,1885754,MTVC-UN,Motive Capital Corp II
11607,1885754,MTVC-WT,Motive Capital Corp II
11608,1909152,KALWW,Kalera Public Ltd Co
11609,1913210,BRSHW,Bruush Oral Care Inc.
